In [1]:
# load environment variables from .env file (requires `python-dotenv`)
from dotenv import load_dotenv
import os

load_dotenv()

True

In [2]:
from langchain_openai import ChatOpenAI


llm = ChatOpenAI(
    model="llama-3.1-8b-instant",
    base_url="https://api.groq.com/openai/v1",
)

llm.invoke("今天是几月几号？").content

'我是一位 AI 模型，目前的时间信息可能已经过时。我的知识截止日期是 2023 年 12 月，如果您需要在当前日期下获取信息，可能会有一定误差。然而根据您的问题，可以知道您问的是 2024 年 7 月 6 日。'

In [ ]:
from langchain.tools import tool
import datetime

@tool
def get_current_date():
    """ 获取今天的日期 """
    return datetime.datetime.today().strftime("%Y-%m-%d")

# 大模型绑定工具
llm_with_tools = llm.bind_tools([get_current_date])

# 工具容器
all_tools = {"get_current_date": get_current_date}

query = "今天是几月几号？"
messages = [query]


ai_msg = llm_with_tools.invoke(messages)
messages.append(ai_msg)

# content 是空。但有 tool calls
print(ai_msg)

# 打印需要调用的工具
print(ai_msg.tool_calls)
if ai_msg.tool_calls:
    for tool_call in ai_msg.tool_calls:
        selected_tool = all_tools[tool_call["name"].lower()]
        tool_msg = selected_tool.invoke(tool_call)
        messages.append(tool_msg)

llm_with_tools.invoke(messages).content

content='' additional_kwargs={'tool_calls': [{'id': 'vfzmtfn9z', 'function': {'arguments': '{}', 'name': 'get_current_date'}, 'type': 'function'}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 208, 'total_tokens': 218, 'completion_tokens_details': None, 'prompt_tokens_details': None, 'queue_time': 0.196183811, 'prompt_time': 0.015766282, 'completion_time': 0.017255887, 'total_time': 0.033022169}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_510c177af0', 'id': 'chatcmpl-ed8280f7-1adf-4708-a4c1-53114e12da3d', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None} id='run--824229e1-2247-4646-a433-25377e92ed3b-0' tool_calls=[{'name': 'get_current_date', 'args': {}, 'id': 'vfzmtfn9z', 'type': 'tool_call'}] usage_metadata={'input_tokens': 208, 'output_tokens': 10, 'total_tokens': 218, 'input_token_details': {}, 'output_token_details': {}}
[{'name': 'get_current_date', 'args': {}, 'id': 'vfzmtfn9z', 'type': 't

'或者，我也可以直接告诉你，今天是7月6号。'

In [8]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts.chat import ChatPromptTemplate

# LCEL 定制一个 chain
prompt = ChatPromptTemplate.from_messages([("human", " 你好，请用下面这种语言回答我的问题 {language}.")])

parser = StrOutputParser()

chain = prompt | llm | parser

# 讲 chain 转换为工具
as_tool = chain.as_tool(name="translate_tool", description="翻译任务")

all_tools = {"translate_tool": as_tool}
print(as_tool.args)

# 绑定工具
llm_with_tools = llm.bind_tools([as_tool])

query = "今天天气真冷，这句话用英语怎么回答？"
messages = [query]

ai_msg = llm_with_tools.invoke(messages)
messages.append(ai_msg)

print(ai_msg.tool_calls)
print(">>>>>>>>>>>>>>>>>")
if ai_msg.tool_calls:
    for tool_call in ai_msg.tool_calls:
        selected_tool = all_tools[tool_call["name"].lower()]
        tool_msg = selected_tool.invoke(tool_call)
        messages.append(tool_msg)

llm_with_tools.invoke(messages).content

{'language': {'title': 'Language', 'type': 'string'}}
[{'name': 'translate_tool', 'args': {'language': 'en', 'name': 'translate_tool', 'text': '今天天气真冷'}, 'id': '6v9ehn08y', 'type': 'tool_call'}]
>>>>>>>>>>>>>>>>>


"That's a kind response."

## Agent

In [ ]:
from langchain.tools import tool
from langchain.agents import initialize_agent, AgentType

@tool(description="获取某个城市的天气")
def get_city_weather(city: str):
    """ 获取某个城市的天气
    Args:
        city: 具体城市
    """
    return  "城市" + city + ", 今天天气不错"

llm2 = ChatOpenAI(
    model="llama3-70b-8192",
    base_url="https://api.groq.com/openai/v1",
)

# 初始化 agent
agent = initialize_agent(
    tools=[get_city_weather],
    llm=llm2,
    agent=AgentType.OPENAI_FUNCTIONS,
    verbose=True,
)

query = "北京今天天气怎么样？"
response = agent.invoke(query)
response



> Entering new AgentExecutor chain...


> Finished chain.


{'input': '北京今天天气怎么样', 'output': ''}